In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# 0. Inicializando Ambiente Django no Notebook

Por enquanto não estou utilizando os dados do banco de dados (e sim o csv presente em csv_data/).

Porém no dia 21/05/25 modifiquei o django_for_jupyter.py para que após a reestruturação dos diretórios os notebooks antigos (e também os dentro de classification) possam acessar o BD futuramente.

In [2]:
import os, sys
sys.path.insert(0, os.path.abspath('..'))

from django_for_jupyter import init_django
init_django('app')

Django initialized for project: app
Project root added to sys.path: /home/esdras-daniel/Documentos/Python/Django/PGM-Text_Classificator


# 1. Load do CSV

In [4]:
notebook_dir = os.getcwd()
csv_file_path = os.path.join(notebook_dir, '..', 'csv_data', 'processed', 'pgm-dataset_clean.csv')
print(f'Caminho do CSV: {csv_file_path}')

df = pd.read_csv(csv_file_path)

Caminho do CSV: /home/esdras-daniel/Documentos/Python/Django/PGM-Text_Classificator/notebooks/../csv_data/processed/pgm-dataset_clean.csv


## 1.1 - Pré-processamento

In [71]:
import nltk
from nltk.corpus import stopwords

# Baixar a lista de stopwords (necessário apenas na primeira vez)
try:
    stopwords.words('portuguese')
except LookupError:
    nltk.download('stopwords')

stopwords_pt = stopwords.words('portuguese')
print(f'Total de stopwords em português: {len(stopwords_pt)}')

Total de stopwords em português: 207


In [72]:
import re
import unicodedata

def preprocess_text(text):

    text = text.lower() # Converter para minúsculas para padronização

    # 1. Remover acentos (nomalização unicode)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

    # 2. Regex
    text = re.sub(r'poder judiciario(?: do)? estado(:? do)? rio grande(:? do)? norte', '', text)
    text = re.sub(r'\d{7}-\d{2}\.\d{4}\.\d{1,2}\.\d{2}\.\d{4}', '[Nº_PROCESSO]', text)
    text = re.sub(r'\(\d{2}\)\s?\d{4,5}-?\d{4}', '[TELEFONE]', text)
    text = re.sub(r'xnone', '', text)

    # 3. Tokenização e remoção de stop words
    words = text.split()
    processed_words = [word for word in words if word not in stopwords_pt]

    # 4. Remover pontuação e caracteres não alfa númericos
    processed_words = [re.sub(r'[^a-z0-9]', '', word) for word in processed_words]
    processed_words = [word for word in processed_words if word] # Remover strings vazias

    return ' '.join(processed_words)

In [73]:
texts_to_embed = df['teorTexto'].apply(preprocess_text).to_numpy()
print(f'Qtd de textos para fazer o embedding: {len(texts_to_embed)}')

Qtd de textos para fazer o embedding: 14926


# 2. Gerando Embeddings

In [83]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
# 1. Parâmetros configuráveis
BERT_MODEL = 'rufimelo/Legal-BERTimbau-base'
AGG_STRATEGY = 'mean'
MAX_TOKENS = 512

# 2. Caminho de saída
embeddings_dir = 'embeddings'
os.makedirs(embeddings_dir, exist_ok=True)
embedding_path = os.path.join(embeddings_dir, f'{BERT_MODEL.replace('/', '-')}_{AGG_STRATEGY}.pt')

# 3. Verificação de existência
if os.path.exists(embedding_path):
    print(f'Embedding já existe em "{embedding_path}". Nenhuma ação necessária.')
else:
    # 4. Carregar modelo e tokenizer
    print(f'Gerando embeddings com o modelo "{BERT_MODEL}" usando agregação "{AGG_STRATEGY}"...')
    tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)
    model = AutoModel.from_pretrained(BERT_MODEL)
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # 5. Função para dividir texto em chunks
    def split_in_chunks(input_ids, max_tokens=MAX_TOKENS):

        # Reservar espaço para [CLS] e [SEP]
        chunk_size = max_tokens - 2
        chunks = [input_ids[i:i+chunk_size] for i in range(0, len(input_ids), chunk_size)]
        return chunks
    
    embeddings_finais = []

    # Obter dimensão dos embeddings (por segurança após carregar modelo)
    with torch.no_grad():
        dummy_input = tokenizer("texto de teste", return_tensors="pt", max_length=MAX_TOKENS, truncation=True)
        dummy_output = model(**dummy_input.to(device))
        embedding_size = dummy_output.last_hidden_state.shape[-1]

    # 6. Embeddings por texto
    for text in tqdm(texts_to_embed):
        if not text.strip():
            zero_emb = torch.zeros(embedding_size)
            embeddings_finais.append(zero_emb)
            continue

        tokens_ids = tokenizer.encode(text, add_special_tokens=False)

        if len(tokens_ids) + 2 > MAX_TOKENS:
            chunks = split_in_chunks(tokens_ids, MAX_TOKENS)
        else:
            chunks = [tokens_ids]

        chunk_embedding = []

        for chunk_ids in chunks:
            # Adiciona os tokens [CLS] e [SEP]
            chunks_ids = [tokenizer.cls_token_ids] + chunk_ids + [tokenizer.sep_token_ids]
            attention_mask = [1] * len(chunk_ids)

            # Converte para tensores
            input_dict = {
                'input_ids': torch.tensor([chunk_ids], device=device),
                'attention_mask': torch.tensor([attention_mask], device=device)
            }

            with torch.no_grad():
                output = model(**input_dict)
                cls_embedding = output.last_hidden_state[:, 0, :]
                chunk_embedding.append(cls_embedding.squeeze(0))
        
        if chunk_embedding:
            chunks_embeddings_tensor = torch.stack(chunk_embedding)
            if AGG_STRATEGY == 'mean':
                emb = torch.mean(chunks_embeddings_tensor, dim=0)
            elif AGG_STRATEGY == 'max':
                emb = torch.max(chunks_embeddings_tensor, dim=0).values
            else:
                raise ValueError("Estratégia inválida. Use 'mean' ou 'max'.")
            embeddings_finais.append(emb.cpu())
        else:
            embeddings_finais.append(torch.zeros(embedding_size))

    # 7. Salvar
    todos_embeddings = torch.stack(embeddings_finais)
    torch.save(todos_embeddings, embedding_path)
    print(f"Embeddings salvos em '{embedding_path}' com shape {todos_embeddings.shape}")


Gerando embeddings com o modelo "rufimelo/Legal-BERTimbau-base" usando agregação "mean"...


Some weights of BertModel were not initialized from the model checkpoint at rufimelo/Legal-BERTimbau-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 14926/14926 [2:46:17<00:00,  1.50it/s]  


Embeddings salvos em 'embeddings/rufimelo-Legal-BERTimbau-base_mean.pt' com shape torch.Size([14926, 768])
